In [ ]:
from sklearn.cluster import KMeans 
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from datetime import date


# Data Getting

In [1524]:
#read a file from this url "https://drive.google.com/file/d/1JNncMKuaXSwuQEp-S6wui4jiDcPGoCkU/view?usp=share_link" and save it as groups_data
groups_data = pd.read_csv("/Users/sebastianrobledo/Documents/Research/3BIO_analytics/info_grupos_merged_separado_instituciones.csv")

# Data describing

In [ ]:
# Show me the column names of groups_data
groups_data.columns

In [ ]:
# count the number of rows and columns in groups_data
groups_data.shape

In [ ]:
# Count the unique values in the column "codigo del grupo"
groups_data["codigo del grupo"].nunique()


In [ ]:
# create a dataframe with the unique values of "codigo del grupo" and other column with the number of times that value appears
groups_data["codigo del grupo"].value_counts() 

In [ ]:
# show me the type of data in each column of groups_data
groups_data.dtypes

In [ ]:
# create a dataframe called estado with column "estado" and remove the string "Categoría " from the values in that column
estado = groups_data["estado"].str.replace("Categoría ","")

In [ ]:
# Create a dataframe from groups_data filtered by the unique values in the column "codigo del grupo"
groups_data_filtered = groups_data.groupby("codigo del grupo").first()

In [ ]:
# Show me the shape of groups_data_filtered
groups_data_filtered

In [ ]:
# Show me the column names of groups_data_filtered
groups_data_filtered.columns

In [ ]:
# Create a dataframe called groups_filtered with the columns "nombre grupo", "estado", "instituciones", "fecha_creacion", "anio"
groups_filtered = groups_data_filtered[["nombre grupo", "estado", "instituciones", "fecha_creacion", "anio"]]

In [ ]:
# Group the values in the column "estado" and count the number of times each value appears, save it as estado_count
estado_count = groups_filtered["estado"].value_counts()

In [ ]:
# Change the values equal to "Categoría 00" to "Grupo Reconocido" in the column "estado" from groups_filtered
groups_filtered["estado"] = groups_filtered["estado"].replace("Categoría 00", "Grupo reconocido")


In [ ]:
# Group the values in the column "estado" and count the number of times each value appears, save it as estado_count_1
estado_count_1 = groups_filtered["estado"].value_counts()

In [ ]:
# Create a dataframe with the unique values of instituciones and other column with the number of times that value appears and save it as instituciones_count. Name the new column "count"
instituciones_count = groups_filtered["instituciones"].value_counts().reset_index(name="count")


In [ ]:
# Create a dataframe with the matrix of the values in the column "instituciones" and the column "estado" and save it as estado_instituciones
estado_instituciones = pd.crosstab(groups_filtered["instituciones"], groups_filtered["estado"])


In [ ]:
# Show me the shape of estado_instituciones
estado_instituciones.shape

In [ ]:
# Show the shape of instituciones_count
instituciones_count.shape

In [ ]:
# Merge the dataframes estado_instituciones and instituciones_count by the column "instituciones" from estado_instituciones and the column "index" from instituciones_count
estado_instituciones = estado_instituciones.merge(instituciones_count, left_index=True, right_on="index")

In [ ]:
# Change the name of the column "estado" to "category" in groups_filtered
groups_filtered = groups_filtered.rename(columns={"estado": "category"})

In [ ]:
# Remove the string "Categoría " from the values in the column "category" from groups_filtered
groups_filtered["category"] = groups_filtered["category"].str.replace("Categoría ","")

In [ ]:
# add to the dataframe named groups_filtered columns  with the values of the column "category"
groups_filtered = pd.get_dummies(groups_filtered, columns=["category"])

In [ ]:
groups_filtered.to_csv("groups.csv")

In [ ]:
#update the dataframe groups_filtered_instituciones with the column "anio" with the min value of the column "anio" from groups_filtered where instituciones is equal to the index of groups_filtered_instituciones
group = groups_filtered["anio"] = groups_filtered.groupby("instituciones")["anio"].min()

In [ ]:
#merge the dataframe "groups_filtered" with the dataframe "gruop" by the column "instutuciones"
groups_filtered = groups_filtered.merge(group, left_on="instituciones", right_on="instituciones")


In [ ]:
#delete column anio_x of dataframe groups_filtered
groups_filtered = groups_filtered.drop(columns=["anio_x"])

In [ ]:
#rename column anio_y of dataframe groups_filtered to min_anio_institucion
groups_filtered = groups_filtered.rename(columns={"anio_y": "min_anio_institucion"})

In [ ]:
# add a column to groups_filtered called "years" with the difference between the current year and the year in the column "anio"
groups_filtered["years"] = date.today().year - groups_filtered["min_anio_institucion"]

In [ ]:
groups_filtered = groups_filtered.groupby('instituciones').agg(
    {
        'min_anio_institucion':'mean',
        'years':'mean',
        'category_A':'sum',
        'category_A1':'sum',
        'category_B':'sum',
        'category_C':'sum',
        'category_Grupo reconocido':'sum'
    }
).reset_index()

In [ ]:
groups_filtered.to_csv("groupsXinstitucion.csv")